1 - As bibliotecas necessárias são pandas, sklearn, nltk, os e zipfile, importadas no Google Colab para manipulação de dados, processamento de texto e treinamento do modelo. Lembrando que para dar certo precisa ter os arquivos já no google colab faça o upload do arquivo que está nesse link : https://archive.ics.uci.edu/dataset/331/sentiment+labelled+sentences

In [1]:
import zipfile
import os

zip_path = "/content/sentiment+labelled+sentences.zip"  # Substitua pelo nome correto do seu arquivo
extract_folder = "/content"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Arquivos extraídos com sucesso!")


Arquivos extraídos com sucesso!


In [2]:
import os

arquivos = os.listdir(extract_folder)
print("Arquivos extraídos:", arquivos)


Arquivos extraídos: ['.config', '__MACOSX', 'sentiment labelled sentences', 'sentiment+labelled+sentences.zip', 'sample_data']


In [15]:
import pandas as pd
import pandas as pd
import os

# Definir os caminhos dos arquivos
folder = "/content/sentiment labelled sentences"
files = ["amazon_cells_labelled.txt", "yelp_labelled.txt", "imdb_labelled.txt"]

# Carregar e concatenar os arquivos
dfs = []
for file_name in files:
    file_path = os.path.join(folder, file_name)
    df = pd.read_csv(file_path, delimiter="\t", header=None, names=["Texto", "Sentimento"])

    dfs.append(df)

# Unir todos os DataFrames
df_final = pd.concat(dfs, ignore_index=True)

# Exibir os dados carregados
df_final

,Texto,Sentimento
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
2743,I just got bored watching Jessice Lange take h...,0
2744,"Unfortunately, any virtue in this film's produ...",0
2745,"In a word, it is embarrassing.",0
2746,Exceptionally bad!,0


2- Utilizamos TfidfVectorizer para transformar os textos brutos em uma matriz numérica, removendo stopwords e normalizando os dados, o que melhora o desempenho do modelo ao eliminar redundâncias e ruídos.

In [18]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

# Vetorização dos textos
vectorizer = TfidfVectorizer(stop_words=stopwords)
X = vectorizer.fit_transform(df_final["Texto"])
y = df_final["Sentimento"]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


3 - O modelo escolhido é o naive bayes, pois é eficiente para classificação de textos e assume independência entre palavras, sendo adequado para análise de sentimentos em conjuntos grandes e esparsos.

4 - Treinamos o modelo dividindo os dados em treino (80%) e teste (20%), ajustando os pesos das palavras com base nas probabilidades condicionalmente dependentes da classe e otimizando a separação entre sentimentos positivos e negativos.

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar e treinar o modelo
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Fazer previsões
y_pred = nb_model.predict(X_test)

# Avaliação do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do Naïve Bayes: {accuracy:.2f}")


Acurácia do Naïve Bayes: 0.81


5 - A avaliação é feita medindo a acurácia no conjunto de teste, indicando a porcentagem de previsões corretas e fornecendo um critério quantitativo para validar a eficácia do modelo na classificação dos textos no nosso caso a Acurácia foi de 81% o que é razoavel

In [32]:
# Frases novas para testar se o resultado vai ser certo ou errado
novas_frases = [
    "This product is amazing! I loved it.",   # Esperado: Positivo
    "Worst experience ever. Total waste of money.",  # Esperado: Negativo
    "A very, very, very slow-moving, aimless movie about a distressed, drifting young man.",
    "This was like the final blow!"
]

# Transformar as frases em vetores TF-IDF
novas_frases_tfidf = vectorizer.transform(novas_frases)

# Fazer a predição
predicoes = nb_model.predict(novas_frases_tfidf)

# Mostrar resultados
for frase, pred in zip(novas_frases, predicoes):
    print(f"Frase: {frase}")
    print(f"Predição: {'Positivo' if pred == 1 else 'Negativo'}")
    print("-" * 50)


Frase: This product is amazing! I loved it.
Predição: Positivo
--------------------------------------------------
Frase: Worst experience ever. Total waste of money.
Predição: Negativo
--------------------------------------------------
Frase: A very, very, very slow-moving, aimless movie about a distressed, drifting young man.
Predição: Negativo
--------------------------------------------------
Frase: This was like the final blow!
Predição: Negativo
--------------------------------------------------


Como podemos ver os resultados foram ótimos